In [1]:
import os

os.environ["PYSYN_CDBS"] = "/home/ruizhew/POSEIDON/inputs/stellar_grids/"
os.environ["POSEIDON_input_data"] = "/home/ruizhew/POSEIDON/inputs/"

In [2]:
from __future__ import absolute_import, unicode_literals, print_function
from POSEIDON.high_res import (
    fast_filter,
    loglikelihood_sysrem,
    get_rot_kernel,
    fit_uncertainties,
    fit_uncertainties_and_remove_outliers,
)
import math, os
import numpy as np
import pickle
import pickle
from scipy import constants
from numba import jit
from astropy.io import fits
from scipy import interpolate
from POSEIDON.core import (
    create_star,
    create_planet,
    define_model,
    make_atmosphere,
    read_opacities,
    wl_grid_constant_R,
    wl_grid_line_by_line,
    compute_spectrum,
)
from POSEIDON.constants import R_Sun
from POSEIDON.visuals import plot_stellar_flux
from POSEIDON.constants import R_J, M_J
import numpy as np
from spectres import spectres
from tqdm import tqdm
from multiprocessing import Pool
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d, median_filter

In [3]:
from scipy.optimize import minimize
from sklearn.decomposition import TruncatedSVD

In [61]:
data = h5py.File("./data/WASP-121b.h5", "r")

In [16]:
chips = ["blue", "redl", "redu"]

for chip in chips:
    flux = data[chip]["flux"][:]
    wl_grid = data[chip]["wl_grid"][:]
    N_order, N_phi, N_wl = flux.shape
    if "flux_blaze_corrected" in data[chip]:
        pass
    else:
        blaze = np.zeros(flux.shape)
        for k in range(N_order):
            order = flux[k]
            median = np.median(order, axis=0)
            order_norm = order / median
            blaze[k] = order_norm

        for det in range(N_order):
            for phi in range(N_phi):
                blaze[det][phi] = median_filter(blaze[det][phi], size=501)

        for det in range(N_order):
            for phi in range(N_phi):
                blaze[det][phi] = gaussian_filter1d(blaze[det][phi], sigma=100)

        flux = flux / blaze

        print("Finish blaze correction")
        data[chip].create_dataset("flux_blaze_corrected", data=flux)

    flux[flux < 0] = 1e-7
    uncertainties = fit_uncertainties(flux)
    print("Finish fit uncertainties")

    for i in range(N_order):
        order = flux[i]
        # out_of_transit_frames = np.concatenate((order[:20], order[-20:]), axis=0)
        median = np.median(order, axis=0)
        order_norm = order / median
        uncertainty = uncertainties[i]

        uncertainty_norm = uncertainty / median

        uncertainties[i] = uncertainty_norm
        flux[i] = order_norm

    residuals, Us = fast_filter(flux, uncertainties, N_iter=15)

    Bs = np.zeros((N_order, N_phi, N_phi))

    for j in range(N_order):
        U = Us[j]
        L = np.diag(1 / np.mean(uncertainties[j], axis=-1))
        B = U @ np.linalg.pinv(L @ U) @ L
        Bs[j] = B

    data[chip].require_dataset("residuals", shape=residuals.shape, dtype=float)
    data[chip]["residuals"][:] = residuals
    data[chip].require_dataset("Bs", shape=Bs.shape, dtype=float)
    data[chip]["Bs"][:] = Bs
    data[chip].require_dataset("uncertainties", shape=uncertainties.shape, dtype=float)
    data[chip]["uncertainties"][:] = uncertainties

blue
0.3112363521515823 238.11804990033775
0.26495334012555566 265.81667728775886
0.30433557090781105 262.7675423238875
0.3014234835155501 263.4181449296351
0.2931882694266263 267.0896506291757
0.308720870594428 262.0207139135431
0.3134401871598662 263.22911483152984
0.3064003220161138 263.47648463089155
0.3068269535954046 266.085395610039
0.29743278273493545 269.9791361314982
0.29661264578671076 268.9852743706458
0.3047211411394326 265.3478716295382
0.2985926798753237 269.94238615111635
0.2869462156411978 274.93524613023806
0.2863247013841429 275.19622301454007
0.29288178644855783 271.7913029485142
0.28957252789178356 273.15488307809966
0.26714120883075176 287.1888768460542
0.2624467613106007 294.24033031406486
0.2626694692582427 292.6013066838359
0.2649026571429537 292.8781587603386
0.26910063387050487 291.1804521651875
0.26661645078710383 293.7618922427317
0.27439768478593607 290.6390998758484
0.2766835525258131 285.7049275422145
0.27520538999296884 288.25821648311904
0.283929618557

In [17]:
for key in data.keys():
    print(data[key])
data.close()

<HDF5 group "/blue" (8 members)>
<HDF5 group "/redl" (8 members)>
<HDF5 group "/redu" (8 members)>


In [32]:
def read(data_all):
    for key in data_all.keys():
        data = data_all[key]
        wl_grid = data["wl_grid"][:]
        residuals = data["residuals"][:]
        Bs = data["Bs"][:]
        phi = data["phi"][:]
        transit_weight = data["transit_weight"][:]
        uncertainties = data["uncertainties"][
            :
        ]  # in case we want to null uncertainties

In [72]:
data_dic = {}

for key in ["blue", "redl", "redu"]:
    data_dic[key] = {}

    def func(name, obj):
        data_dic[key][name] = obj[:]

    data[key].visititems(func)

In [73]:
data_dic["blue"].keys()

dict_keys(['Bs', 'flux', 'flux_blaze_corrected', 'phi', 'residuals', 'transit_weight', 'uncertainties', 'wl_grid'])